## Field to extract
- Timestamp (TimeCreated)
- TargetUserName
- IpAddress
- LogonType
- Status/ Substatus
- EventID

In [1]:
import Evtx
import lxml
import tqdm
import pandas

## Parse Security.evtx into a table

Plan:
- Read data/Security.evtx
- Loop through events
- Extract fields (timestamp, target user, ip, event id, logon type)
- Build list of dicts
- Convert to DataFrame


In [2]:
import os
os.path.exists("../data/Security.evtx")


True

In [3]:
from pathlib import Path

cwd = Path.cwd()
project_root = cwd if (cwd / "data").exists() else cwd.parent
evtx_path = project_root / "data" / "Security.evtx"
print(evtx_path)  # see the full resolved path


C:\Users\user\Desktop\University\Projects\basic-log-analysis\data\Security.evtx


In [4]:
from pathlib import Path

cwd = Path.cwd()
project_root = cwd if (cwd / "data").exists() else cwd.parent
evtx_path = project_root / "data" / "Security.evtx"
print(evtx_path)  # see the full resolved path


C:\Users\user\Desktop\University\Projects\basic-log-analysis\data\Security.evtx


In [5]:
# OPEN AND PEEK AT EVTX (prints the first 5 events as XML)

import os
from Evtx.Evtx import Evtx

# because the notebook is in /scripts, we go one folder up then into /data
evtx_path = "../data/Security.evtx"

# sanity check
if not os.path.exists(evtx_path):
    raise FileNotFoundError(f"Couldn't find: {evtx_path}")

# open the EVTX and print a few records as XML
with Evtx(evtx_path) as evtx:
    for i, record in enumerate(evtx.records()):
        print(f"\n=== EVENT {i+1} XML ===")
        print(record.xml())   # <-- this prints raw XML
        if i >= 4:            # just 5 events max
            break



=== EVENT 1 XML ===
<Event xmlns="http://schemas.microsoft.com/win/2004/08/events/event"><System><Provider Name="Microsoft-Windows-Security-Auditing" Guid="{54849625-5478-4994-a5ba-3e3b0328c30d}"></Provider>
<EventID Qualifiers="">4907</EventID>
<Version>0</Version>
<Level>0</Level>
<Task>13568</Task>
<Opcode>0</Opcode>
<Keywords>0x8020000000000000</Keywords>
<TimeCreated SystemTime="2025-10-17 19:08:03.494673+00:00"></TimeCreated>
<EventRecordID>340565</EventRecordID>
<Correlation ActivityID="" RelatedActivityID=""></Correlation>
<Execution ProcessID="4" ThreadID="3936"></Execution>
<Channel>Security</Channel>
<Computer>Dell-G3</Computer>
<Security UserID=""></Security>
</System>
<EventData><Data Name="SubjectUserSid">S-1-5-18</Data>
<Data Name="SubjectUserName">DELL-G3$</Data>
<Data Name="SubjectDomainName">WORKGROUP</Data>
<Data Name="SubjectLogonId">0x00000000000003e7</Data>
<Data Name="ObjectServer">Security</Data>
<Data Name="ObjectType">File</Data>
<Data Name="ObjectName">C:\Wi

In [6]:
# --- IMPORTS & PATHS ---
from pathlib import Path
from Evtx.Evtx import Evtx
from lxml import etree
import pandas as pd

# detect project root whether notebook runs in / or /scripts
cwd = Path.cwd()
project_root = cwd if (cwd / "data").exists() else cwd.parent
evtx_path = project_root / "data" / "Security.evtx"

print("Using EVTX at:", evtx_path)
assert evtx_path.exists(), "Security.evtx not found. Put it in data/Security.evtx"


Using EVTX at: C:\Users\user\Desktop\University\Projects\basic-log-analysis\data\Security.evtx


In [7]:
# --- XML HELPERS ---

# parse an event's XML string into an lxml element
def parse_xml(xml_str: str):
    return etree.fromstring(bytes(xml_str, encoding="utf-8"))

# xpath that ignores namespaces using local-name()
def xp(node, path):
    # returns list of matching elements/values
    return node.xpath(path)

def first_text(node, path, default=None):
    vals = xp(node, path)
    if not vals:
        return default
    # if it is attribute or text node, it's already a string
    v = vals[0]
    # lxml returns strings for @attr and text(); if it's an element, get its text
    if isinstance(v, str):
        return v
    return (v.text or default)

def first_attr(node, path, default=None):
    vals = xp(node, path)
    return vals[0] if vals else default


In [8]:
# --- PARSE EVTX INTO ROWS ---

rows = []
limit_preview = 0   # set to e.g. 200 for a quick preview; leave 0 for full parse

with Evtx(str(evtx_path)) as evtx:
    for i, rec in enumerate(evtx.records()):
        xml_str = rec.xml()
        root = parse_xml(xml_str)

        # System-level fields
        event_id = first_text(root, "//*[local-name()='EventID']/text()", default=None)
        system_time = first_attr(root, "//*[local-name()='TimeCreated']/@SystemTime", default=None)

        # EventData fields
        target_user = first_text(root, "//*[local-name()='Data' and @Name='TargetUserName']/text()", default=None)
        ip          = first_text(root, "//*[local-name()='Data' and @Name='IpAddress']/text()", default=None)
        logon_type  = first_text(root, "//*[local-name()='Data' and @Name='LogonType']/text()", default=None)
        status      = first_text(root, "//*[local-name()='Data' and @Name='Status']/text()", default=None)
        sub_status  = first_text(root, "//*[local-name()='Data' and @Name='SubStatus']/text()", default=None)

        rows.append({
            "system_time": system_time,
            "event_id": event_id,
            "target_user": target_user,
            "ip": ip,
            "logon_type": logon_type,
            "status": status,
            "sub_status": sub_status
        })

        if limit_preview and i+1 >= limit_preview:
            break

df = pd.DataFrame(rows)
print("Parsed rows:", len(df))
df.head()


Parsed rows: 29738


,system_time,event_id,target_user,ip,logon_type,status,sub_status
0,2025-10-17 19:08:03.494673+00:00,4907,None,None,None,None,None
1,2025-10-17 19:08:03.495354+00:00,4907,None,None,None,None,None
2,2025-10-17 19:08:03.532240+00:00,4907,None,None,None,None,None
3,2025-10-17 19:08:03.532904+00:00,4907,None,None,None,None,None
4,2025-10-17 19:08:03.537617+00:00,4907,None,None,None,None,None


In [9]:
# --- NORMALIZE ---
# timestamp
df["timestamp"] = pd.to_datetime(df["system_time"], errors="coerce", utc=True)

# numeric casts (nullable)
df["event_id"]   = pd.to_numeric(df["event_id"], errors="coerce").astype("Int64")
df["logon_type"] = pd.to_numeric(df["logon_type"], errors="coerce").astype("Int64")

# keep + order columns
df = df[["timestamp","event_id","target_user","ip","logon_type","status","sub_status"]]

# quick sanity
print("Rows:", len(df))
print("Time range:", df["timestamp"].min(), "→", df["timestamp"].max())
df.head()


Rows: 29738
Time range: 2025-10-17 19:08:03.494673+00:00 → 2025-11-01 14:42:44.786701+00:00


,timestamp,event_id,target_user,ip,logon_type,status,sub_status
0,2025-10-17 19:08:03.494673+00:00,4907,None,None,<NA>,None,None
1,2025-10-17 19:08:03.495354+00:00,4907,None,None,<NA>,None,None
2,2025-10-17 19:08:03.532240+00:00,4907,None,None,<NA>,None,None
3,2025-10-17 19:08:03.532904+00:00,4907,None,None,<NA>,None,None
4,2025-10-17 19:08:03.537617+00:00,4907,None,None,<NA>,None,None


In [10]:
# --- FILTER & SAVE ---
failed  = df[df["event_id"] == 4625].copy()
success = df[df["event_id"] == 4624].copy()

data_dir = project_root / "data"
failed.to_csv(data_dir / "failed_logons.csv", index=False)
success.to_csv(data_dir / "success_logons.csv", index=False)

print("Failed rows:", len(failed), "→ saved to", data_dir / "failed_logons.csv")
print("Success rows:", len(success), "→ saved to", data_dir / "success_logons.csv")


Failed rows: 5 → saved to C:\Users\user\Desktop\University\Projects\basic-log-analysis\data\failed_logons.csv
Success rows: 1071 → saved to C:\Users\user\Desktop\University\Projects\basic-log-analysis\data\success_logons.csv


In [11]:
print("Total events:", len(df))
print("4625 failed:", len(failed))
print("4624 success:", len(success))

if len(failed):
    print("Failed time range:", failed["timestamp"].min(), "→", failed["timestamp"].max())
    print("Failed logon_type values:", sorted([x for x in failed["logon_type"].dropna().unique()]))
    print("Failed with empty IP:", failed["ip"].isna().sum() + (failed["ip"] == "").sum())
failed.head()


Total events: 29738
4625 failed: 5
4624 success: 1071
Failed time range: 2025-11-01 14:39:40.579376+00:00 → 2025-11-01 14:39:47.608610+00:00
Failed logon_type values: [np.int64(2)]
Failed with empty IP: 0


,timestamp,event_id,target_user,ip,logon_type,status,sub_status
29723,2025-11-01 14:39:40.579376+00:00,4625,Daniele Monaco,127.0.0.1,2,0xc000006d,0xc000006a
29724,2025-11-01 14:39:43.931002+00:00,4625,Daniele Monaco,127.0.0.1,2,0xc000006d,0xc000006a
29725,2025-11-01 14:39:45.380770+00:00,4625,Daniele Monaco,127.0.0.1,2,0xc000006d,0xc000006a
29726,2025-11-01 14:39:46.617188+00:00,4625,Daniele Monaco,127.0.0.1,2,0xc000006d,0xc000006a
29727,2025-11-01 14:39:47.608610+00:00,4625,Daniele Monaco,127.0.0.1,2,0xc000006d,0xc000006a


In [12]:
# --- CHARTS ---
import matplotlib.pyplot as plt

screens_dir = project_root / "screenshots"
screens_dir.mkdir(exist_ok=True)

# 6.1 failed by user (top 10)
if len(failed):
    top_users = (failed["target_user"]
                 .fillna("UNKNOWN")
                 .value_counts()
                 .head(10))

    plt.figure()
    top_users.plot(kind="bar", title="Failed logons by user (top 10)")
    plt.tight_layout()
    out1 = screens_dir / "failed_by_user.png"
    plt.savefig(out1, dpi=150)
    plt.close()
    print("Saved:", out1)

# 6.2 logon type distribution in failed
if len(failed):
    logon_counts = failed["logon_type"].fillna(-1).astype(int).value_counts().sort_index()

    plt.figure()
    logon_counts.plot(kind="bar", title="Failed logons by LogonType (-1 = missing)")
    plt.tight_layout()
    out2 = screens_dir / "logon_type_distribution.png"
    plt.savefig(out2, dpi=150)
    plt.close()
    print("Saved:", out2)

# 6.3 failures by hour-of-day
if len(failed):
    hours = failed["timestamp"].dt.tz_convert("UTC").dt.hour  # keep consistent
    hour_counts = hours.value_counts().sort_index()

    plt.figure()
    hour_counts.plot(kind="bar", title="Failed logons by hour (UTC)")
    plt.tight_layout()
    out3 = screens_dir / "failed_by_hour.png"
    plt.savefig(out3, dpi=150)
    plt.close()
    print("Saved:", out3)


Saved: C:\Users\user\Desktop\University\Projects\basic-log-analysis\screenshots\failed_by_user.png
Saved: C:\Users\user\Desktop\University\Projects\basic-log-analysis\screenshots\logon_type_distribution.png
Saved: C:\Users\user\Desktop\University\Projects\basic-log-analysis\screenshots\failed_by_hour.png


In [13]:
print("🔁 Parsing events…")
print("Using:", evtx_path)
print("Parsed rows:", len(df))
from IPython.display import display
display(df.head())

🔁 Parsing events…
Using: C:\Users\user\Desktop\University\Projects\basic-log-analysis\data\Security.evtx
Parsed rows: 29738


,timestamp,event_id,target_user,ip,logon_type,status,sub_status
0,2025-10-17 19:08:03.494673+00:00,4907,None,None,<NA>,None,None
1,2025-10-17 19:08:03.495354+00:00,4907,None,None,<NA>,None,None
2,2025-10-17 19:08:03.532240+00:00,4907,None,None,<NA>,None,None
3,2025-10-17 19:08:03.532904+00:00,4907,None,None,<NA>,None,None
4,2025-10-17 19:08:03.537617+00:00,4907,None,None,<NA>,None,None


## Normalization

- Convert 'SystemTime' → 'timestamp' (Timezone-aware datetime, UTC OK).
- Cast 'event_id' and 'logon_type' to integers (handle blanks safely).
- Standardize colums to: 'timestamp', 'event_id', 'target_user', 'logon_type', 'status', 'sub_status'
- Keep only those columns in the working table.
- Note: 'ip' may be empty for local logons (expected).

In [14]:
print("Rows:", len(df))
print("Dtypes:\n", df.dtypes)
print("Time range:", df["timestamp"].min(), "→", df["timestamp"].max())
display(df.head())

Rows: 29738
Dtypes:
 timestamp      datetime64[ns, UTC]
event_id                     Int64
target_user                 object
ip                          object
logon_type                   Int64
status                      object
sub_status                  object
dtype: object
Time range: 2025-10-17 19:08:03.494673+00:00 → 2025-11-01 14:42:44.786701+00:00


,timestamp,event_id,target_user,ip,logon_type,status,sub_status
0,2025-10-17 19:08:03.494673+00:00,4907,None,None,<NA>,None,None
1,2025-10-17 19:08:03.495354+00:00,4907,None,None,<NA>,None,None
2,2025-10-17 19:08:03.532240+00:00,4907,None,None,<NA>,None,None
3,2025-10-17 19:08:03.532904+00:00,4907,None,None,<NA>,None,None
4,2025-10-17 19:08:03.537617+00:00,4907,None,None,<NA>,None,None


In [15]:
df["timestamp"].min(), df["timestamp"].max()

(Timestamp('2025-10-17 19:08:03.494673+0000', tz='UTC'),
 Timestamp('2025-11-01 14:42:44.786701+0000', tz='UTC'))

In [16]:
print("Saving CSVs…")
failed.to_csv(project_root / "data" / "failed_logons.csv", index=False)
success.to_csv(project_root / "data" / "success_logons.csv", index=False)
print("Saved:", project_root / "data" / "failed_logons.csv")
print("Saved:", project_root / "data" / "success_logons.csv")


Saving CSVs…
Saved: C:\Users\user\Desktop\University\Projects\basic-log-analysis\data\failed_logons.csv
Saved: C:\Users\user\Desktop\University\Projects\basic-log-analysis\data\success_logons.csv


In [17]:
out1 = screens_dir / "failed_by_user.png"
plt.savefig(out1, dpi=150)
print("Saved:", out1)
print("Saved:", out2)
print("Saved:", out3)

Saved: C:\Users\user\Desktop\University\Projects\basic-log-analysis\screenshots\failed_by_user.png
Saved: C:\Users\user\Desktop\University\Projects\basic-log-analysis\screenshots\logon_type_distribution.png
Saved: C:\Users\user\Desktop\University\Projects\basic-log-analysis\screenshots\failed_by_hour.png


<Figure size 640x480 with 0 Axes>

In [18]:
print("Total rows:", len(df))
print("Failed rows (4625):", len(failed))
print("Success rows (4624):", len(success))


Total rows: 29738
Failed rows (4625): 5
Success rows (4624): 1071


In [19]:
failed["logon_type"].dropna().value_counts()

logon_type
2    5
Name: count, dtype: Int64

## Filter & Save outputs

- Split into:
  - failed: `event_id == 4625`
  - success: `event_id == 4624`
- Save into CSV files:
  - `data/failed_logons.csv`
  - `data/success_logons.csv`

## Sanity checks after parsing

- Count total rows
- Count rows where 'event_id == 4625'
- Count rows where 'event_id == 4624'
- Inspect data range (min/max timestamp)
- List unique 'logon_type' values seen in failed logons
- View first few rows ('head()') to validate columns
- Count failed rows where 'ip' is empty

In [20]:
print("Min timestamp:", df["timestamp"].min())
print("Max timestamp:", df["timestamp"].max())

Min timestamp: 2025-10-17 19:08:03.494673+00:00
Max timestamp: 2025-11-01 14:42:44.786701+00:00
